In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import os
from PIL import Image
from sklearn import svm, linear_model, ensemble, neighbors, tree
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
datasets = "/content/drive/MyDrive/datasets"

# Lister les images dans les sous-répertoires
no_tumor_images = os.listdir(os.path.join(datasets, 'no'))
yes_tumor_images = os.listdir(os.path.join(datasets, 'yes'))

dataset=[]
label=[]

INPUT_SIZE=224

print('No Tumor: ', len(no_tumor_images))
print('Tumor: ',len(yes_tumor_images))

No Tumor:  1500
Tumor:  1500


In [5]:
for image_name in no_tumor_images:
    if image_name.split('.')[-1].lower() == 'jpg':
        image_path = os.path.join(datasets, 'no', image_name)
        try:
            image = cv2.imread(image_path)
            if image is not None:
                image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                image = image.resize((INPUT_SIZE, INPUT_SIZE))
                dataset.append(np.array(image))
                label.append(0)
            else:
                print(f"Could not read image: {image_path}")
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")

for image_name in yes_tumor_images:
    if image_name.split('.')[-1].lower() == 'jpg':
        image_path = os.path.join(datasets, 'yes', image_name)
        try:
            image = cv2.imread(image_path)
            if image is not None:
                image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                image = image.resize((INPUT_SIZE, INPUT_SIZE))
                dataset.append(np.array(image))
                label.append(1)
            else:
                print(f"Could not read image: {image_path}")
        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")

In [6]:
dataset = np.array(dataset)
label = np.array(label)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=0)

In [8]:
X_train = X_train/255.0
X_test = X_test/255.0

In [9]:


# Définir le modèle CNN
model = Sequential()

# 1ère couche de convolution et de pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2ème couche de convolution et de pooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3ème couche de convolution et de pooling
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Aplatir les résultats pour le réseau fully connected
model.add(Flatten())

# Couche fully connected avec dropout
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Couche de sortie
model.add(Dense(1, activation='sigmoid'))

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Définir les callbacks
checkpoint = ModelCheckpoint('best_model_cnn.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entraîner le modèle
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])


Epoch 1/20
75/75 [==============================] - ETA: 0s - loss: 0.5341 - accuracy: 0.7525

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


75/75 [==============================] - 324s 4s/step - loss: 0.5341 - accuracy: 0.7525 - val_loss: 0.4496 - val_accuracy: 0.7933
Epoch 2/20
75/75 [==============================] - 346s 5s/step - loss: 0.3493 - accuracy: 0.8517 - val_loss: 0.3424 - val_accuracy: 0.8650
Epoch 3/20
75/75 [==============================] - 324s 4s/step - loss: 0.2529 - accuracy: 0.8971 - val_loss: 0.3087 - val_accuracy: 0.8900
Epoch 4/20
75/75 [==============================] - 332s 4s/step - loss: 0.1787 - accuracy: 0.9350 - val_loss: 0.2476 - val_accuracy: 0.9150
Epoch 5/20
75/75 [==============================] - 357s 5s/step - loss: 0.1404 - accuracy: 0.9558 - val_loss: 0.2424 - val_accuracy: 0.9167
Epoch 6/20
75/75 [==============================] - 359s 5s/step - loss: 0.1106 - accuracy: 0.9658 - val_loss: 0.1779 - val_accuracy: 0.9367
Epoch 7/20
75/75 [==============================] - 348s 5s/step - loss: 0.0847 - accuracy: 0.9762 - val_loss: 0.1681 - val_accuracy: 0.9450
Epoch 8/20
75/75 [======

In [11]:
# Charger le meilleur modèle sauvegardé
model.load_weights('best_model_cnn.h5')

# Évaluer le modèle
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


19/19 [==============================] - 25s 1s/step - loss: 0.1994 - accuracy: 0.9667
Test Accuracy: 96.67%
